### PanStarrs

In [ ]:
from astropy.table import Table

In [ ]:
import matplotlib.pyplot as plt
import arya

In [ ]:
from astropy import units as u

In [ ]:
import numpy as np

In [ ]:
from pathlib import Path

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../../imaging")

from phot_utils import xmatch, get_atm_extinction
from astropy.nddata import CCDData

In [ ]:
import tomllib

In [ ]:
panstarrs = Table.read("../../survey_data/yasone2_panstarrs.fits")

In [ ]:
cat_all = Table.read("../yasone2/allcolours.cat")

In [ ]:
cat_good = cat_all[(cat_all["RFLAGS_WEIGHT"] <2 ) & (cat_all["RFLAGS"] < 8)]

In [ ]:
np.unique(cat_all["RFLAGS"])

In [ ]:
panstarrs_filt = (panstarrs["qualityFlag"] & 4).reshape(-1) > 0

for filt in ["g", "r", "i"]:
    panstarrs_filt &= panstarrs[f"{filt}Flags"].reshape(-1) & 2 == 0
    panstarrs_filt &= panstarrs[f"{filt}Flags"].reshape(-1) & 1 == 0
    # panstarrs_filt &= panstarrs[f"{filt}MeanPSFMagNpt"].reshape(-1) > 2
    


In [ ]:
panstarrs_good = panstarrs[panstarrs_filt]

In [ ]:
plt.figure()
plt.scatter(cat_good["ALPHA_J2000"], cat_good["DELTA_J2000"], s=2, lw=0)

plt.scatter(panstarrs["ra"], panstarrs["dec"], s=1, lw=0)
plt.xlim(np.min(cat_good["ALPHA_J2000"]) - 0.01, np.max(cat_good["ALPHA_J2000"]) + 0.01)
plt.ylim(np.min(cat_good["DELTA_J2000"]) - 0.01, np.max(cat_good["DELTA_J2000"]) + 0.01)

In [ ]:
xmatch_idx, xmatch_dist, xmatch_filt_panstarrs = xmatch(cat_good["ALPHA_J2000"], cat_good["DELTA_J2000"], panstarrs_good["ra"].reshape(-1), panstarrs_good["dec"].reshape(-1), 1*u.arcsec)

In [ ]:
panstarrs_xmatch = panstarrs_good[xmatch_idx]
osiris_x_panstarrs = cat_good[xmatch_filt_panstarrs]

In [ ]:
idx_aper = 5

for filt in ["g", "r", "i"]:
    cat_good[f"mag_{filt}_panstarrs"] = panstarrs_xmatch[f"{filt}MeanApMag"].reshape(-1)
    cat_good[f"mag_{filt}_err_panstarrs"] = panstarrs_xmatch[f"{filt}MeanApMagErr"].reshape(-1)

    cat_good[f"mag_{filt}_panstarrs"][~xmatch_filt_panstarrs] = np.nan
    cat_good[f"mag_{filt}_err_panstarrs"][~xmatch_filt_panstarrs] = np.nan
        

In [ ]:
cat = cat_good[xmatch_filt_panstarrs]

In [ ]:
len(cat), len(cat_all)

In [ ]:
plt.figure()
plt.scatter(cat["ALPHA_J2000"]/u.degree, cat["DELTA_J2000"]/u.degree, s=0.3)

plt.scatter(panstarrs_xmatch["ra"], panstarrs_xmatch["dec"])
plt.scatter(cat["ALPHA_J2000"]/u.degree, cat["DELTA_J2000"]/u.degree, s=1)
# plt.xlim(-0.1, 0.1)
# plt.ylim(-0.1, 0.1)

In [ ]:
panstarrs

In [ ]:
plt.hist(cat["FWHM_WORLD"].to("arcsec"))

In [ ]:

fig, axs = plt.subplots(2, 3, sharex=True, sharey="row", figsize=(7, 4), gridspec_kw={"hspace": 0, "wspace": 0})
c = None
for i in range(3):
    plt.sca(axs[0][i])
    cs = arya.COLORS[0] #np.log10(cat["FWHM_WORLD"].to("arcsec") / u.arcsec)

    filt = ["g", "r", "i"][i]
    mag0 = cat[f"{filt.upper()}MAG"]
    mag_ps = cat[f"mag_{filt}_panstarrs"]
    mag_ps_err = cat[f"mag_{filt}_err_panstarrs"]
    plt.scatter(mag0, mag_ps, s=1,  c=cs, alpha=0.1)
    plt.xlim(24, 17)
    plt.ylim(24, 17)
    plt.plot([26, 10], [26, 10], color="black", zorder=-1)
    if i == 0:
        plt.ylabel(f"mag (PS)")
    
    plt.sca(axs[1][i])
    
    yerr = np.maximum(mag_ps_err, 0)
    # plt.errorbar(mag0[1:-1:20], (mag_ps-mag0)[1:-1:20], yerr=yerr[1:-1:20], fmt=".", capsize=0, )
    c = plt.scatter(mag0, mag_ps - mag0, s=1, c=cs, alpha=0.1)

    plt.ylim(3, -3)
    plt.axhline(0, color="black", zorder=-1)
    
    plt.xlabel(f"{filt} (osiris)")
    if i == 0:
        plt.ylabel("residual")


plt.tight_layout()